In [1]:
import trimesh
import numpy as np

In [2]:
lod_4_mesh = trimesh.load_mesh("/home/jeans/win/aaaJAIST/resources/LOD_data_50/1/lod2.obj")
lod_3_mesh = trimesh.load_mesh("/home/jeans/win/aaaJAIST/resources/LOD_data_50/1/lod1.obj")

In [3]:

def center_mesh_origin_by_bbox(mesh):
    """
    Center the mesh at the origin by translating it.
    
    Parameters:
    mesh (trimesh.Trimesh): The mesh to center.
    
    Returns:
    trimesh.Trimesh: The centered mesh.
    """
    # get min/max of x,y,z
    min_corner, max_corner = mesh.bounds  # each is (3,)
    box_center = (min_corner + max_corner) * 0.5

    # translate so bounding‐box center → origin
    mesh.apply_translation(-box_center)
    
    return mesh

def center_mesh_origin_by_centroid(mesh):
    """
    Center the mesh at the origin by translating it to its centroid.
    
    Parameters:
    mesh (trimesh.Trimesh): The mesh to center.
    
    Returns:
    trimesh.Trimesh: The centered mesh.
    """
    centroid = mesh.centroid
    mesh.apply_translation(-centroid)
    
    return mesh

#before centering
scene = trimesh.Scene([lod_4_mesh, lod_3_mesh])
scene.show()


def register_and_transfrom(mesh, other):
    """
    Register two meshes and display them in a scene.
    
    Parameters:
    mesh (trimesh.Trimesh): The first mesh to register.
    other (trimesh.Trimesh): The second mesh to register against the first.
    """
    matrix, cost = trimesh.registration.mesh_other(
        mesh,
        other,
        samples=3000,
        scale=False,
        icp_first=1,
        icp_final=30
    )
    
    print(f"Alignment cost: {cost:.6f}")
    
    mesh_copy = mesh.copy()
    return mesh_copy.apply_transform(matrix)


In [4]:
mesh_tran = register_and_transfrom(lod_4_mesh, lod_3_mesh)

#show after registration
scene = trimesh.Scene([mesh_tran, lod_3_mesh])

Alignment cost: 0.001037


In [5]:
scene.show()

In [6]:
lod_4_mesh_centered_bbox = center_mesh_origin_by_bbox(lod_4_mesh)
lod_3_mesh_centered_bbox = center_mesh_origin_by_bbox(lod_3_mesh)

mesh_tran = register_and_transfrom(lod_4_mesh_centered_bbox, lod_3_mesh_centered_bbox)

#show after registration
scene = trimesh.Scene([mesh_tran, lod_3_mesh_centered_bbox])
scene.show()

Alignment cost: 0.001104


In [7]:

lod_4_mesh_centered_centroid = center_mesh_origin_by_centroid(lod_4_mesh)
lod_3_mesh_centered_centroid = center_mesh_origin_by_centroid(lod_3_mesh)


mesh_tran = register_and_transfrom(lod_4_mesh_centered_centroid, lod_3_mesh_centered_centroid)
#show after registration
scene = trimesh.Scene([mesh_tran, lod_3_mesh_centered_centroid])
scene.show()

Alignment cost: 0.001114
